Install the transformer library

In [ ]:
pip install transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 4.4 MB 9.5 MB/s 
     |████████████████████████████████| 596 kB 47.2 MB/s 
     |████████████████████████████████| 101 kB 9.1 MB/s 
     |████████████████████████████████| 6.6 MB 51.2 MB/s 
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13


In [ ]:
pip install inputs

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
import requests
from transformers import BertTokenizer, BertForNextSentencePrediction
import torch

In [ ]:
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
model=BertTokenizer.from_pretrained('bert-base-uncased') 

Downloading:   0%|          | 0.00/226k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/28.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/570 [00:00<?, ?B/s]

Getting the data

In [ ]:
data=requests.get('https://github.com/jamescalam/transformers/blob/main/data/text/meditations/clean.txt')

In [ ]:
text=data.text.split('\n')

In [ ]:
text

['',
 '',
 '',
 '',
 '',
 '',
 '',
 '<!DOCTYPE html>',
 '<html lang="en" data-color-mode="auto" data-light-theme="light" data-dark-theme="dark" data-a11y-animated-images="system">',
 '  <head>',
 '    <meta charset="utf-8">',
 '  <link rel="dns-prefetch" href="https://github.githubassets.com">',
 '  <link rel="dns-prefetch" href="https://avatars.githubusercontent.com">',
 '  <link rel="dns-prefetch" href="https://github-cloud.s3.amazonaws.com">',
 '  <link rel="dns-prefetch" href="https://user-images.githubusercontent.com/">',
 '  <link rel="preconnect" href="https://github.githubassets.com" crossorigin>',
 '  <link rel="preconnect" href="https://avatars.githubusercontent.com">',
 '',
 '',
 '',
 '  <link crossorigin="anonymous" media="all" integrity="sha512-ksfTgQOOnE+FFXf+yNfVjKSlEckJAdufFIYGK7ZjRhWcZgzAGcmZqqArTgMLpu90FwthqcCX4ldDgKXbmVMeuQ==" rel="stylesheet" href="https://github.githubassets.com/assets/light-92c7d381038e.css" /><link crossorigin="anonymous" media="all" integrity="s

In [ ]:
data[:3]

TypeError: ignored

In [ ]:
text[51].split('.') 

['    <script crossorigin="anonymous" defer="defer" integrity="sha512-sAeL7y7Tbq5g07u3/0pJIS9IegVGfOVwfX7LQfKPna4UYzOQtcjfAJnwXSHWbAG7jSy4jQKtsukdRuQLX8JrGQ==" type="application/javascript" data-module-id="',
 '/chunk-voting',
 'js" data-src="https://github',
 'githubassets',
 'com/assets/chunk-voting-b0078bef',
 'js"></script>']

In [ ]:
bag = [item for sentence in text for item in sentence.split('.') if item != '']
bag_size = len(bag)

In [ ]:
import random

sentence_a = []
sentence_b = []
label = []

for paragraph in text:
    sentences = [
        sentence for sentence in paragraph.split('.') if sentence != ''
    ]
    num_sentences = len(sentences)
    if num_sentences > 1:
        start = random.randint(0, num_sentences-2)
        # 50/50 whether is IsNextSentence or NotNextSentence
        if random.random() >= 0.5:
            # this is IsNextSentence
            sentence_a.append(sentences[start])
            sentence_b.append(sentences[start+1])
            label.append(0)
        else:
            index = random.randint(0, bag_size-1)
            # this is NotNextSentence
            sentence_a.append(sentences[start])
            sentence_b.append(bag[index])
            label.append(1)

In [ ]:
for i in range(3):
    print(label[i])
    print(sentence_a[i] + '\n---')
    print(sentence_b[i] + '\n')

0
githubassets
---
com">

1
  <link rel="dns-prefetch" href="https://avatars
---
githubassets

0
s3
---
amazonaws



In [ ]:
inputs = tokenizer(sentence_a, sentence_b, return_tensors='pt', max_length=512, truncation=True, padding='max_length')

NameError: ignored

In [ ]:
inputs.keys()

NameError: ignored

In [ ]:
inputs.token_type_ids[0]

NameError: ignored

In [ ]:
inputs['labels'] = torch.LongTensor([label]).T

NameError: ignored

In [ ]:
inputs.labels[:10]

NameError: ignored

In [ ]:
class MeditationsDataset(torch.utils.data.Dataset):
    def __init__(self, encodings):
        self.encodings = encodings
    def __getitem__(self, idx):
        return {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
    def __len__(self):
        return len(self.encodings.input_ids)

In [ ]:
dataset = MeditationsDataset(inputs)

NameError: ignored

In [ ]:
loader = torch.utils.data.DataLoader(dataset, batch_size=16, shuffle=True)

NameError: ignored

In [ ]:
device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')
# and move our model over to the selected device
model.to(device

SyntaxError: ignored

In [ ]:
from transformers import AdamW

# activate training mode
model.train()
# initialize optimizer
optim = AdamW(model.parameters(), lr=5e-6)

NameError: ignored

In [ ]:
from tqdm import tqdm  # for our progress bar

epochs = 2

for epoch in range(epochs):
    # setup loop with TQDM and dataloader
    loop = tqdm(loader, leave=True)
    for batch in loop:
        # initialize calculated gradients (from prev step)
        optim.zero_grad()
        # pull all tensor batches required for training
        input_ids = batch['input_ids'].to(device)
        attention_mask = batch['attention_mask'].to(device)
        token_type_ids = batch['token_type_ids'].to(device)
        labels = batch['labels'].to(device)
        # process
        outputs = model(input_ids, attention_mask=attention_mask,
                        token_type_ids=token_type_ids,
                        labels=labels)
        # extract loss
        loss = outputs.loss
        # calculate loss for every parameter that needs grad update
        loss.backward()
        # update parameters
        optim.step()
        # print relevant info to progress bar
        loop.set_description(f'Epoch {epoch}')
        loop.set_postfix(loss=loss.item())


NameError: ignored